# Analyse de trajectoire longue proche d'un mur rigide (avec Cython)

Code origine : $\mathrm{\color{red}{RigidWallOverdampedLangevin3D\_cython.py}}$

Auteur : $\mathrm{\color{purple}{Élodie ~MILLAN}}$

Date : 05/10/2021

## 0. Déscription du problème :


Simulation et analyse d'une trajectoire brownienne proche d'une paroie rigide. 

Une particule de rayon $a$, de masse volumique $\rho$, plongé dans un fluide de viscosité $\eta_0$, soumise au poid $P=\Delta m g=(m_\mathrm{F}-m_\mathrm{P}) g$, où $m_\mathrm{F}$ est la masse de fluide déplacé par la poussé d'Archimède et $m_\mathrm{P}$, la masse de la particule.

La particule, à une altitude $z$ de la paroie, est soumis à la répulation éléctrostatique de la paroie, $R_\mathrm{elec}$ :
\begin{equation}
    R_\mathrm{elec} = \frac{4 k_\mathrm{b} T}{l_\mathrm{d}} exp( - \frac{z}{l_\mathrm{d}})
\end{equation}
où $k_\mathrm{b}$ est la constante de Boltzman et $l_\mathrm{d}$ la longueur de Debye.

La présence de la paroie induit une viscosité effective du fluide qui dépend de l'altitude $z$ de la particule $\eta(z)$ dans les dirrection collinaire $\eta_\|$ et transverse du mur $\eta_\perp$ :

\begin{equation}
    \eta_\| (z) = \frac{6 \pi \eta_0 a}{\left( 1 - \frac{9a}{16(z+a)} +  \left( \frac{a}{8(z+a)} \right)^3 - \left( \frac{45a}{256(z+a)} \right)^4 - \left( \frac{a}{16(z+a)} \right)^5 \right)},
\end{equation}
obtenue dans l'approximation de Libchaber, et,
\begin{equation}
    \eta_\perp (z) = 6 \pi \eta_0 a ~ \frac{6z^2+9az+2a^2 }{6z^2 + 2az},
\end{equation}
avec la formule de Padé.

On concidère les effets inertiels négligeable, on peux alors écrire l'équation de $\color{blue}{\mathrm{Langevin~sur~ amortie}}$ : 

\begin{equation}
    x(t+\tau) = x(t) + (P + R_\mathrm{elec} + F_\mathrm{corr} ) \frac{\tau}{\gamma (z)} + \sqrt{2D(z)} ~ dB_t ~ \tau
\end{equation}
où $D(z) = \frac{k_\mathrm{b}T}{6 \pi \eta_i(z) a}$, avec $i=\{ \|,\perp \}$.

Une force corrective $F_\mathrm{corr}$ est ajouté pour ajuster le cas sur-amortie.

\begin{equation}
    F_\mathrm{corr} = k_b T \frac{D_z'}{D_z} = k_b T \frac{42 R z^2 + 24 R^2z + 4R^2}{36 z^4 + 66Rz^3 + 30 R^2 z^2 + 4R^3 z}.
\end{equation}


In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import gc
# import seaborn as sns
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import scipy
import sys
import pyximport

pyximport.install(setup_args={'include_dirs': np.get_include()},reload_support=True)
sys.path.append(r"/home/e.millan/Documents/Stage2020-Nageurs-actifs-proche-de-parois-deformable/OptimizedCython")
from RigidWallOverdampedLangevin3D_cython import RigidWallOverdampedLangevin3D 

from mpl_toolkits import mplot3d
from scipy.signal import correlate
from scipy.optimize import curve_fit

import sys
import glob
from tqdm import tqdm
from scipy.optimize import curve_fit
from tqdm import trange
import matplotlib as mpl
mpl.rcParams['figure.dpi']=140
mpl.rcParams['xtick.direction']="in"
mpl.rcParams['ytick.direction']="in"
mpl.rcParams['lines.markeredgecolor']="k"
mpl.rcParams['lines.markeredgewidth']=0.2
mpl.rcParams['lines.markersize'] = 10
from matplotlib import rc
rc('font', family='serif')
rc('text', usetex=True)
rc("xtick", labelsize=20)
rc("ytick", labelsize=20)
from sys import getsizeof

/home/e.millan/miniconda3/envs/cy3/lib/python3.9/site-packages/Cython/Compiler/Main.py:346: FutureWarning: Cython directive 'language_level' not set, using '3str' for now (Py3). This has changed from earlier releases! File: /home/e.millan/Documents/Stage2020-Nageurs-actifs-proche-de-parois-deformable/OptimizedCython/RigidWallOverdampedLangevin3D_cython.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)

Error compiling Cython file:
------------------------------------------------------------
...

        :param space: Choose between "bulk" and "wall".
        :param plot: Plot MSD is True.
        :param output: Return {tau, MSD(tau)} is True.
        """
    if axis == "x":
      ^
------------------------------------------------------------

/home/e.millan/Documents/Stage2020-Nageurs-actifs-proche-de-parois-deformable/OptimizedCython/RigidWallOverdampedLangevin3D_cython.pyx:132:7: undeclared name not builtin: axis

Error compiling Cython file:
-----------------------------------

ImportError: Building module RigidWallOverdampedLangevin3D_cython failed: ["distutils.errors.CompileError: command '/usr/bin/gcc' failed with exit code 1\n"]

In [ ]:
# sys.path.append(r"/home/e.millan/Documents/Stage2020-Nageurs-actifs-proche-de-parois-deformable/Results")
# from Functions_Analysis import plot_traj, MSD, PDF, P_z_wall

## 1. Calcul trajectoire avec code origine

In [ ]:
dt = 1/60
Nt = 50000000
a = 1.5e-6
rho = 1050
r0 = (0., 0., 1.5e-6)


simu = RigidWallOverdampedLangevin3D(dt=dt, Nt=Nt, R=a, rho=rho, x0=r0)
simu.trajectory()

Rs = np.zeros([Nt, 3])

Rs[:,0] = simu.x
Rs[:,1] = simu.y
Rs[:,2] = simu.z

In [ ]:
print("t_num ~", np.around(simu.Nt*simu.dt,1), "s")

## 2. Plot trajectoire

In [ ]:
# plot_traj(simu.t,Rs)

## 3. plot PDF

### 3.a) $P_\mathrm{eq}(z)$
La PDF théorique des altitudes $z$ à l'équilibre s'écrit :
\begin{equation}
    P_\mathrm{eq} (z) = N ~ \exp(- B ~ e^{- \frac{z}{l_\mathrm{d}} } - \frac{z}{l_\mathrm{b}} ),
\end{equation}
où $N$ est une constante de normalisation, $l_\mathrm{d}$ la longueur de Debye et $l_\mathrm{b}$ la longueur de Boltzman, $B=4.8$ un facteur déterminer expérimentalement $\color{red}{\mathrm{(À ~vérifier ~!!)}}$.

In [ ]:
plt.axis([-1e-7, 3.2e-6, 5e3, 2e6])
simu.PDF("z", N_tau=10, space="wall", bins=100, plot=True)

On peut calculer la valeur moyenne des coeficients de diffusion collinaire et transverse à l'aide de $P_\mathrm{eq}(z)$ :
\begin{equation}
    \langle D_i \rangle_z = \int_{-\infty}^{+\infty} D_i ~ P_\mathrm{eq}(z) \mathrm{d}z .
\end{equation}

In [ ]:
# zth = np.linspace(1e-7, 3e-6, 200) ## Une liste de z
# pdf_theo = P_z_wall(zth, 1, 4.8, simu.lD, simu.lB) ## P_eq(z)
# Peq_z = pdf_theo/np.trapz(pdf_theo, zth) ## Normalisation par N

# """ 
#     Sur l'axe Z
# """
# Dz = (simu.kb*simu.T) / (simu._gamma_z(zth)) ## D_\perp (z)
# mean_Dz_theo = np.trapz(Dz * Peq_z, zth) ## <D_\perp (z)>
# print("<Dz>_theo/D_0 = {:.2f}".format(mean_Dz_theo/(simu.kb * simu.T / simu.gamma)))

# """ 
#     Sur l'axe X
# """
# Dx = (simu.kb*simu.T) / (simu._gamma_xy(zth)) ## D_|| (z)
# mean_Dx_theo = np.trapz(Dx * Peq_z, zth)  ## <D_|| (z)>
# print("<Dx>_theo/D_0 = {:.10f}".format(mean_Dx_theo/(simu.kb * simu.T / simu.gamma)))

### 3.b) $P(\Delta x_i, \tau)$, $x_i = \{ x,y,z \}$.

La PDF des déplacement en $x$, $y$ et $z$ sont données par la relation: 


\begin{equation}
    P(\Delta x_i, \tau) = \frac{1}{N} \int_{-\infty}^{+\infty} P(D_i) \exp \left( -\frac{\Delta x_i^2}{4 D_i \tau} \right) ~\mathrm{d}D_i,
\end{equation}
où le déplacement est calculé à partir du retard $\tau$ tel que $\Delta x_i = x_i(t+\tau)-x_i(t)$ et $N$ est le facteur de normalisation.

La PDF des coeficients de disctributioin est donnée par :

\begin{equation}
    P(D_i) = \int_{-\infty}^{+\infty} D_i(z)~ P_\mathrm{eq}(z)~ \mathrm{d}z.
\end{equation}

Lorsque $\tau \rightarrow +\infty$, c'est-à-dire lorsque $\tau \gg \tau_\mathrm{c}$, la PDF des déplacements en $z$ devient :

\begin{equation}
    \lim_{\tau \rightarrow +\infty} P(\Delta z, \tau) = \int_{-\infty}^{+\infty} P_\mathrm{eq}(z+ \Delta z)~ P_\mathrm{eq}(z) ~\mathrm{d}z,
\end{equation}

où $\tau_\mathrm{c}$ est le temps caractéristique pour atteindre l'equilibre thermodynamique et est donnée par :

\begin{equation}
    \tau_\mathrm{c} = \frac{l_\mathrm{B} a}{D_0}.
\end{equation}

$\mathrm{\color{purple}{SOURCE~ :~ thèse~ Maxime, ~page ~90}}$.

In [ ]:
bins = 100
n_tau = 1
plt.axis([-6, 6,1e0, 1e7])
simu.PDF("dx", N_tau=n_tau, space="wall", bins=bins, plot=True)

In [ ]:
plt.axis([-6, 6,1e0, 1e7])
simu.PDF("dy", N_tau=n_tau, space="wall", bins=100, plot=True)

In [ ]:
plt.axis([-10, 10,1e0, 1e8])
simu.PDF("dz", N_tau=n_tau, space="wall", bins=100, plot=True)

In [ ]:
plt.axis([-10, 10,0.5e-1, 2e6])
simu.PDF("dz", N_tau=3000, space="wall", bins=100, plot=True)

## 4. PLot MSD

In [ ]:
# MSD("z", Rs, simu.dt, simu.Nt, D_mean=mean_Dz_theo, lD=simu.lD, lB=simu.lB, space="wall")

In [ ]:
simu.MSD(axis="z", plot=True, output=False)

## 5. Cumulant ordre 4

Le calcul du cumulant d'ordre $4$ s'effectue à l'aide de la relation :

\begin{equation}
    C^{(4)} = \frac{1}{4!}\left( \langle z(t)-z(t+\tau) \rangle^4 - 3 \langle (z(t)-z(t+\tau)^2 \rangle^2 \right).
\end{equation}

Théoriquement, au temps court on a :
\begin{equation}
    \lim_{\tau \rightarrow 0}~C^{(4)} = \frac{\tau ^2}{2}\left( \langle D_{\|}^2 \rangle - \langle D_{\|} \rangle^2 \right),
\end{equation}
 et au temps long :
 
\begin{equation}
    \lim_{\tau \rightarrow +\infty}~C^{(4)} = C^{(4)}_0 \tau + C^{(4)}_1.
\end{equation}

 où $C^{(4)}_0$ et $C^{(4)}_1$ sont à determiner numériquement à l'aide du code d'Arthur Alexandre (cf. cumul4.py et note).

In [ ]:
simu.Cumulant4(axis="x", plot=True, output=False)